# NOP NOP Trajectories Bug Testing

Includes functionality to:
1. Read encounter information straight from the bin files
2. Simulate encounter files
3. Visualize those simulations (if not small_simulation_output())

**Make sure you are in a Julia 1.1 Env!
And that you have aircraftshapes.sty in repo**

In [1]:
include("src/Xr.jl");

In [44]:
# Generate new encounters from within the Jupyter notebook
@time cd("../UAMEncounterGen") do
    include("generate_nop_nop_encs.jl")
    cp(filename1, joinpath("../XrSim/data_files", basename(filename1)), force=true)
    cp(filename2, joinpath("../XrSim/data_files", basename(filename2)), force=true)
end

┌ Warning: encounter_files/nop_nop_UstarUstar_flip_test_1z.bin already exists... overwriting!
└ @ Main C:\Users\mossr\Code\sisl\ACAS Xr\UAMEncounterGen\generate_nop_nop_encs.jl:15


  5.834089 seconds (3.74 M allocations: 891.554 MiB, 4.72% gc time)


"../XrSim/data_files/nop_nop_UstarUstar_flip_test_2z.bin"

### 1. Simulating the encounter files

Make sure this notebook is within the UAMEncounterGen repo. Make sure that your encounter files are in the "data_files" folder within UAMEncounterGenerator. 

When running a simulation, to see alerts or nmacs, must do sim.sim_out = small_simulation_output()

When running a simulation, to be able to visualize it, do not do a small simulation

sim 1 : trajectory 1 equipped as ac1

sim 2 : trajectory 1 equipped as ac2

sim 3 : trajectory 2 equipped as ac2

sim 4 : trajectory 2 equipped as ac1

In [50]:
# Settings for all aircraft
run_vertical_logic = false
is_responsive = false
sim_out_for_plots = true # set to false to see the alert counts (true for plotting sim output)

encounter_file1 = joinpath("data_files", basename(filename1))
encounter_file2 = joinpath("data_files", basename(filename2))

sim1 = simulation()

sim_out_for_plots ? nothing : sim1.sim_out = small_simulation_output()

sim1.enc_file = encounter_file1
if run_vertical_logic
    sim1.acs[1] = uam_vert(q_file = "data_files/test.bin")
else
    sim1.acs[1] = uam_speed(q_file = "data_files/test_speed_120.bin")
    sim1.acs[1].perform_scaling = true
end

# turn responsive to false when wanting to check if trajectories are being properly generated
sim1.acs[1].responsive = is_responsive

xr_sim!(sim1);

In [51]:
sim2 = simulation()
sim_out_for_plots ? nothing : sim2.sim_out = small_simulation_output()
sim2.enc_file = encounter_file2
if run_vertical_logic
    sim2.acs[2] = uam_vert(q_file = "data_files/test.bin")
else
    sim2.acs[2] = uam_speed(q_file = "data_files/test_speed_120.bin")
    sim2.acs[2].perform_scaling = true
end
sim2.acs[2].responsive = is_responsive
xr_sim!(sim2);

In [52]:
sim3 = simulation()
sim_out_for_plots ? nothing : sim3.sim_out = small_simulation_output()
sim3.enc_file = encounter_file1
if run_vertical_logic
    sim3.acs[2] = uam_vert(q_file = "data_files/test.bin")
else
    sim3.acs[2] = uam_speed(q_file = "data_files/test_speed_120.bin")
    sim3.acs[2].perform_scaling = true
end
sim3.acs[2].responsive = is_responsive
xr_sim!(sim3);

In [53]:
sim4 = simulation()
sim_out_for_plots ? nothing : sim4.sim_out = small_simulation_output()
sim4.enc_file = encounter_file2
if run_vertical_logic
    sim4.acs[1] = uam_vert(q_file = "data_files/test.bin")
else
    sim4.acs[1] = uam_speed(q_file = "data_files/test_speed_120.bin")
    sim4.acs[1].perform_scaling = true
end
sim4.acs[1].responsive = is_responsive
xr_sim!(sim4);

In [49]:
if sim_out_for_plots
    @warn "Rerun with `set_out_for_plots = true`"
else
    println(sim1.sim_out.alerts)
    println(sim2.sim_out.alerts)
    println()
    println(sim3.sim_out.alerts)
    println(sim4.sim_out.alerts)
end

69
69

61
61


### 2. Visualizing the simulated encounter sets

Make sure the simulation you want to visualize does not have sim.sim_out = small_simulation_output()

ac1 = UAM icon

ac2 = quad icon

In [14]:
include("visualization/UAMEncounterViewer.jl");

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-15939659123941140862\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-10184236264480009242\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-15939659123941140862\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

In [54]:
encounter_viewer(sim1.sim_out, int_type=:quad, alert_type=:speed)

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["File Name"], Dict{Symbol,Any}(:className => "interact ",:style => Dict{Any,Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol,Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(:attributes => Dict{Any,Any}(:type => "text",Symbol("data-bind") => "value: value, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}"),:placeholder => "",:className => "input ",:style => Dict{Any,Any}()))], Dict{Symbol,Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[], Dict{Symbol,Any}())], Dict{Symbol,Any}(:className => "interact-flex-row-right"))], Dict{Symbol,Any}(:className => "interact-flex-row interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 2 listeners. Value:
0, nothing),"value" => (Observable{String} with 1 listeners. Value:
"myFile.pdf", nothing)), Set(String[]), nothing, Asset[Asset("js", "knockout", "C:\\Users\\mossr\\.julia\\packages\\Knockout\\IP1uR\\src\\..\\assets\\knockout.js"), Asset("js", "knockout_punches", "C:\\Users\\mossr\\.julia\\packages\\Knockout\\IP1uR\\src\\..\\assets\\knockout_punches.js"), Asset("js", nothing, "C:\\Users\\mossr\\.julia\\packages\\InteractBase\\sOe2Z\\src\\..\\assets\\all.js"), Asset("css", nothing, "C:\\Users\\mossr\\.julia\\packages\\InteractBase\\sOe2Z\\src\\..\\assets\\style.css"), Asset("css", nothing, "C:\\Users\\mossr\\.julia\\packages\\Interact\\SbgIk\\src\\..\\assets\\bulma_confined.min.css")], Dict{Any,Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"value" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"value\"]()) ? (this.valueFromJulia[\"value\"]=true, this.model[\"value\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set(AbstractConnection[]), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x0000000037a61cd0, Task (runnable) @0x0000000037a61cd0), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"changes\":WebIO.getval({\"name\":\"changes\",\"scope\":\"8822613857983482835\",\"id\":\"16539304506985167953\",\"type\":\"observable\"}),\"value\":WebIO.getval({\"name\":\"value\",\"scope\":\"8822613857983482835\",\"id\":\"12730551331302370299\",\"type\":\"observable\"})};\n    var self = this;\n    functio

In [55]:
encounter_viewer(sim4.sim_out, int_type=:quad, alert_type=:speed)

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["File Name"], Dict{Symbol,Any}(:className => "interact ",:style => Dict{Any,Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol,Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(:attributes => Dict{Any,Any}(:type => "text",Symbol("data-bind") => "value: value, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}"),:placeholder => "",:className => "input ",:style => Dict{Any,Any}()))], Dict{Symbol,Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[], Dict{Symbol,Any}())], Dict{Symbol,Any}(:className => "interact-flex-row-right"))], Dict{Symbol,Any}(:className => "interact-flex-row interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 2 listeners. Value:
0, nothing),"value" => (Observable{String} with 1 listeners. Value:
"myFile.pdf", nothing)), Set(String[]), nothing, Asset[Asset("js", "knockout", "C:\\Users\\mossr\\.julia\\packages\\Knockout\\IP1uR\\src\\..\\assets\\knockout.js"), Asset("js", "knockout_punches", "C:\\Users\\mossr\\.julia\\packages\\Knockout\\IP1uR\\src\\..\\assets\\knockout_punches.js"), Asset("js", nothing, "C:\\Users\\mossr\\.julia\\packages\\InteractBase\\sOe2Z\\src\\..\\assets\\all.js"), Asset("css", nothing, "C:\\Users\\mossr\\.julia\\packages\\InteractBase\\sOe2Z\\src\\..\\assets\\style.css"), Asset("css", nothing, "C:\\Users\\mossr\\.julia\\packages\\Interact\\SbgIk\\src\\..\\assets\\bulma_confined.min.css")], Dict{Any,Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"value" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"value\"]()) ? (this.valueFromJulia[\"value\"]=true, this.model[\"value\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set(AbstractConnection[]), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x0000000039702290, Task (runnable) @0x0000000039702290), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"changes\":WebIO.getval({\"name\":\"changes\",\"scope\":\"7190275675436902400\",\"id\":\"3136596188685881195\",\"type\":\"observable\"}),\"value\":WebIO.getval({\"name\":\"value\",\"scope\":\"7190275675436902400\",\"id\":\"15650369346432861074\",\"type\":\"observable\"})};\n    var self = this;\n    function

### 3. Reading straight from the encounter files

In [ ]:
open("data_files/nop_nop_UstarUstar_flip_test_1z.bin", "r") do f
    dt = read(f, Float64)
    num_steps = read(f, Int64)
    # For each encounter
    for i = 1:2
        # Make an encounter out of info
        curr_phys_state = read_phys_state(f)
        ẍ = read_vector(f, num_steps)
        ÿ = read_vector(f, num_steps)
        z̈ = read_vector(f, num_steps)

        #@show curr_phys_state
        #@show ẍ
        #@show ÿ
        #@show z̈
        #@show length(ẍ)
    end
end
